# Unsupervised analysis of [Colorectal cancer subtyping](https://www.nature.com/articles/nm.3967) data
We are using only the TCGA cohort (n = 604) for simplicity. Get [clinical data](https://www.synapse.org/#!Synapse:syn2325330) and [gene expression data](https://www.synapse.org/#!Synapse:syn2325328) from Synapse

In [ ]:
# !pip install scikit-learn umap-learn networkx python-louvain

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import umap

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, MDS

from scipy.spatial.distance import pdist, squareform

## CRC TCGA cohort
### Gene expression data
The gene expression file is **tab-separated**, so we need to specify `sep = '\t'`

We also **transpose** (flip) the patients and genes with `.T`

In [ ]:
crc_gene_data = pd.read_csv('datasets/TCGACRC_expression-merged.zip', index_col = 0, header = 0, sep = '\t', compression = 'zip')
crc_gene_data = crc_gene_data.T
crc_gene_data.head(2)

### Clinical data

In [ ]:
crc_clin_data = pd.read_csv('datasets/TCGACRC_clinical-merged.tsv', index_col = 0, header = 0, sep = '\t')
crc_clin_data.head(2)

### Join two tables
`axis = 1` means that we are joining the columns

`join = 'inner'` means that we are only taking patients that exist in both tables

In [ ]:
crc_data = pd.concat([crc_clin_data, crc_gene_data], axis = 1, join = 'inner')
print(crc_data.shape)
crc_data.head(2)

### Update gene expression table to contain only patients with clinical data
Confirm the number of rows

In [ ]:
crc_gene_data = crc_gene_data.loc[crc_data.index]
print(crc_gene_data.shape)

## Dimensionality reduction
### PCA
For PCA, we will standardize the gene expression profile. A simple way is to subtract by mean and divide by standard deviation. Pandas automatically expand the `.mean()` and `.std()` dataframes for us

In [ ]:
crc_gene_data_std = (crc_gene_data - crc_gene_data.mean()) / crc_gene_data.std()
crc_gene_data_std.head(2)

### Confirm that the means are close to zeros and the standard deviations are all ones

In [ ]:
print(crc_gene_data_std.mean())

In [ ]:
print(crc_gene_data_std.std())

### Another way to standardize the data is via scikit-learn [sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) function
However, the result is a numpy array, not a pandas dataframe

Notice to `.fit()` and `.transform()` pattern

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(crc_gene_data)
crc_gene_data_std_v2 = scaler.transform(crc_gene_data)

print(crc_gene_data_std_v2)

### Perform PCA

In [ ]:
crc_gene_pca = PCA(random_state = 4649).fit(crc_gene_data_std)
crc_gene_pca_embed = crc_gene_pca.transform(crc_gene_data_std)

### Visualize explained variance ratios
Truncated at the first 50

In [ ]:
plt.bar(range(1, 51), crc_gene_pca.explained_variance_ratio_[:50])
plt.xlabel('PC'); plt.ylabel('explained variance');

### Visualize data on PC1 vs PC2

In [ ]:
plt.figure(figsize = (5, 5))
plt.scatter(crc_gene_pca_embed[:, 0], crc_gene_pca_embed[:, 1])
plt.xlabel('PC1'); plt.ylabel('PC2');

### Color by some clinical variables

In [ ]:
crc_data[crc_clin_data.columns].head(2)

In [ ]:
print(pd.unique(crc_data['tStage']))
print(pd.unique(crc_data['nStage']))
print(pd.unique(crc_data['mStage']))
print(pd.unique(crc_data['microsatelite']))
print(pd.unique(crc_data['tumorLocation']))

In [ ]:
def view_embed_clinical_cat(embed, feature, value_order = None):
    plt.figure(figsize = (5, 5))

    no_value = pd.isna(crc_data[feature])
    plt.scatter(embed[no_value, 0], embed[no_value, 1], c = 'lightgray')
    
    if value_order is None:
        value_order = sorted(pd.unique(crc_data[feature].loc[~pd.isna(crc_data[feature])]))
    
    for value in value_order:
        patients = crc_data[feature] == value
        plt.scatter(embed[patients, 0], embed[patients, 1], label = value, alpha = 0.5)
    
    plt.xlabel('PC1'); plt.ylabel('PC2')
    plt.title('colored by ' + feature)
    plt.legend()
    plt.show()

In [ ]:
view_embed_clinical_cat(crc_gene_pca_embed, 'tStage', ['T0', 'T1', 'T2', 'T3', 'T4', 'T4a', 'T4b'])
view_embed_clinical_cat(crc_gene_pca_embed, 'mStage', ['M0', 'M1', 'M1a', 'M1b', 'Mx'])
view_embed_clinical_cat(crc_gene_pca_embed, 'tumorLocation')
view_embed_clinical_cat(crc_gene_pca_embed, 'microsatelite', ['MSS', 'MSI-L', 'MSI-H', 'Indeterminate'])

### View loadings on the first two PCA components
Limited to only 50 genes

In [ ]:
plt.figure(figsize = (13, 4))
plt.bar(range(50), crc_gene_pca.components_[0][:50])
plt.xticks(range(50), labels = crc_gene_data.columns[:50], rotation = 90)
plt.ylabel('PC1 loading')
plt.show()

plt.figure(figsize = (13, 4))
plt.bar(range(50), crc_gene_pca.components_[1][:50])
plt.xticks(range(50), labels = crc_gene_data.columns[:50], rotation = 90)
plt.ylabel('PC2 loading')
plt.show()

## Let's try calculating similarity between patients using correlation instead
Use `crc_gene_data.T` because correlations are calculated between columns

In [ ]:
crc_corr_matrix = crc_gene_data.T.corr(method = 'pearson')
display(crc_corr_matrix.head(3))

### To embed with correlation, we need PCoA instead of PCA
PCoA is a special case of `sklearn.manifold.MDS`

Input `1 - crc_corr_matrix` because correlation and similarity are inverse

In [ ]:
pcoa = MDS(n_components = 2, random_state = 4649, dissimilarity = 'precomputed')
crc_gene_pcoa_embed = pcoa.fit_transform(1 - crc_corr_matrix)

In [ ]:
view_embed_clinical_cat(crc_gene_pcoa_embed, 'tStage', ['T0', 'T1', 'T2', 'T3', 'T4', 'T4a', 'T4b'])
view_embed_clinical_cat(crc_gene_pcoa_embed, 'mStage', ['M0', 'M1', 'M1a', 'M1b', 'Mx'])
view_embed_clinical_cat(crc_gene_pcoa_embed, 'tumorLocation')
view_embed_clinical_cat(crc_gene_pcoa_embed, 'microsatelite', ['MSS', 'MSI-L', 'MSI-H', 'Indeterminate'])

## Moving on to more advanced techniques
### t-SNE
Try several perplexity values

Use **%%time** to measure the amount of CPU time used

In [ ]:
%%time
perplexities = [5, 15, 25, 50, 100]

plt.figure(figsize = (13, 3))

for i, k in enumerate(perplexities, start = 1):
    plt.subplot(1, 5, i)
    crc_gene_tsne_embed = TSNE(n_components = 2, perplexity = k, random_state = 4649).fit_transform(crc_gene_data_std)
    plt.scatter(crc_gene_tsne_embed[:, 0], crc_gene_tsne_embed[:, 1], s = 2)
    plt.xlabel('tSNE1'); plt.ylabel('tSNE2')
    plt.title('perplexity = ' + str(k))

plt.tight_layout()
plt.show()

### UMAP
Try several neighbor values

In [ ]:
%%time
neighbers = [5, 15, 25, 50, 100]

plt.figure(figsize = (13, 3))

for i, n in enumerate(neighbers, start = 1):
    plt.subplot(1, 5, i)
    umap_embed = umap.UMAP(n_components = 2, n_neighbors = n, random_state = 4649).fit_transform(crc_gene_data_std)
    plt.scatter(umap_embed[:, 0], umap_embed[:, 1], s = 2)
    plt.xlabel('UMAP1'); plt.ylabel('UMAP2')
    plt.title('n_neighbors = ' + str(n))

plt.tight_layout()
plt.show()

## Let's use UMAP with n_neighbors = 50

In [ ]:
crc_gene_umap_embed = umap.UMAP(n_components = 2, n_neighbors = 50, random_state = 4649).fit_transform(crc_gene_data_std)

In [ ]:
view_embed_clinical_cat(crc_gene_umap_embed, 'tStage', ['T0', 'T1', 'T2', 'T3', 'T4', 'T4a', 'T4b'])
view_embed_clinical_cat(crc_gene_umap_embed, 'mStage', ['M0', 'M1', 'M1a', 'M1b', 'Mx'])
view_embed_clinical_cat(crc_gene_umap_embed, 'tumorLocation')
view_embed_clinical_cat(crc_gene_umap_embed, 'microsatelite', ['MSS', 'MSI-L', 'MSI-H', 'Indeterminate'])

### Effect of UMAP's min_dist

In [ ]:
min_dists = [0, 0.25, 0.75, 1]

plt.figure(figsize = (12, 3))

for i, m in enumerate(min_dists, start = 1):
    plt.subplot(1, 4, i)
    umap_embed = umap.UMAP(n_components = 2, n_neighbors = 50, min_dist = m, random_state = 4649).fit_transform(crc_gene_data_std)
    plt.scatter(umap_embed[:, 0], umap_embed[:, 1], s = 2)
    plt.xlabel('UMAP1'); plt.ylabel('UMAP2')
    plt.title('min_dist = ' + str(m))

plt.tight_layout()
plt.show()

### Finalize UMAP embedding with n_neighbor = 50 and min_dist = 0.25

In [ ]:
crc_gene_final_embed = umap.UMAP(n_components = 2, n_neighbors = 50, min_dist = 0.25, random_state = 4649).fit_transform(crc_gene_data_std)

In [ ]:
plt.figure(figsize = (5, 5))
plt.scatter(crc_gene_final_embed[:, 0], crc_gene_final_embed[:, 1])
plt.xlabel('UMAP1'); plt.ylabel('UMAP2');

## Who are the patients that are tightly grouped together?
Let's select them by UMAP coordinates

In [ ]:
plt.figure(figsize = (5, 5))
plt.scatter(crc_gene_final_embed[:, 0], crc_gene_final_embed[:, 1])
plt.axis([2, 4, -0.5, 0.5])
plt.xlabel('UMAP1'); plt.ylabel('UMAP2');

In [ ]:
mysterious_patients = crc_gene_data.index[(crc_gene_final_embed[:, 0] <= 3.5) & (crc_gene_final_embed[:, 1] >= -0.3)]
print(mysterious_patients)

In [ ]:
crc_clin_data.loc[mysterious_patients]

# Clustering
We will try k-mean, hierarchical, DBSCAN, and some network techniques

`silhouette_score` and `calinski_harabasz_score` are quality scores for a clustering result

In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score, calinski_harabasz_score

## k-mean
`KMeans.inertia_` record the variance

`n_init = 5` makes the clustering algorithm repeats 5 times

In [ ]:
kmean_silhouette = []
kmean_caha = []
kmean_inertia = []

for k in range(2, 10):
    kmean = KMeans(n_clusters = k, random_state = 4649, n_init = 5).fit(crc_gene_data_std)
    kmean_inertia.append(kmean.inertia_)
    
    kmean_silhouette.append(silhouette_score(crc_gene_data_std, kmean.labels_))
    kmean_caha.append(calinski_harabasz_score(crc_gene_data_std, kmean.labels_))

### All evaluations point toward two clusters

In [ ]:
plt.figure(figsize = (12, 3))

plt.subplot(1, 3, 1)
plt.plot(range(2, 10), kmean_inertia)
plt.xlabel('number of clusters'); plt.ylabel('Inertia'); plt.title('Elbow method')

plt.subplot(1, 3, 2)
plt.plot(range(2, 10), kmean_silhouette)
plt.xlabel('number of clusters'); plt.ylabel('Silhouette'); plt.title('Find maximum')

plt.subplot(1, 3, 3)
plt.plot(range(2, 10), kmean_caha)
plt.xlabel('number of clusters'); plt.ylabel('Calinski-Harabasz'); plt.title('Find maximum')

plt.tight_layout()
plt.show()

### Visualize the location of the two clusters

In [ ]:
def view_clusters(labels, embed = crc_gene_final_embed):
    plt.figure(figsize = (5, 5))
    
    for k in np.unique(labels):
        group = (labels == k)
        plt.scatter(embed[group, 0], embed[group, 1], s = 20, label = 'cluster ' + str(k))

    plt.xlabel('UMAP1'); plt.ylabel('UMAP2'); plt.legend()

In [ ]:
kmean_cluster = KMeans(n_clusters = 2, random_state = 4649).fit_predict(crc_gene_data)
view_clusters(kmean_cluster)
view_clusters(kmean_cluster, crc_gene_pcoa_embed)

## Agglomerative / Hierarchical clustering
Default with `average` linkage

In [ ]:
hierarchical_silhouette = []
hierarchical_caha = []

for k in range(2, 10):
    hierarchical = AgglomerativeClustering(n_clusters = k, affinity = 'euclidean', linkage = 'average').fit(crc_gene_data_std)  
    hierarchical_silhouette.append(silhouette_score(crc_gene_data_std, hierarchical.labels_))
    hierarchical_caha.append(calinski_harabasz_score(crc_gene_data_std, hierarchical.labels_))

plt.figure(figsize = (8, 3))
plt.subplot(1, 2, 1)
plt.plot(range(2, 10), hierarchical_silhouette)
plt.xlabel('Number of clusters'); plt.ylabel('Silhouette')

plt.subplot(1, 2, 2)
plt.plot(range(2, 10), hierarchical_caha)
plt.xlabel('Number of clusters'); plt.ylabel('Calinski-Harabasz')

plt.tight_layout()
plt.show()

### Silhouette suggests 2 clusters while CH suggests 3

In [ ]:
hierarchical2 = AgglomerativeClustering(n_clusters = 2, affinity = 'euclidean', linkage = 'average').fit_predict(crc_gene_data_std)
hierarchical3 = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage = 'average').fit_predict(crc_gene_data_std)

In [ ]:
view_clusters(hierarchical2)
view_clusters(hierarchical2, crc_gene_pcoa_embed)

In [ ]:
view_clusters(hierarchical3)
view_clusters(hierarchical3, crc_gene_pcoa_embed)

## DBSCAN
This method doesn't need the number of cluster to be specified. But we need to tune `epsilon` instead

Consider values of `epsilon` rangaing from 0.2 to 0.75

In [ ]:
eps_start = 0.2
eps_step = 0.05
eps = np.arange(eps_start, eps_step * 11 + eps_start, eps_step)

plt.figure(figsize = (12, 10))

for i, e in enumerate(eps, start = 1):
    plt.subplot(3, 4, i)
    dbscan = DBSCAN(eps = e, metric = 'correlation', min_samples = 10)
    predicted = dbscan.fit_predict(crc_gene_data_std)
    
    for k in sorted(np.unique(predicted)):
        filt = predicted == k
        plt.scatter(crc_gene_final_embed[filt, 0], crc_gene_final_embed[filt, 1], label = 'cluster ' + str(k), s = 10)

    plt.xlabel('UMAP1'); plt.ylabel('UMAP2'); plt.legend()
    plt.title('epsilon = ' + str(e)[:4])

plt.tight_layout()
plt.show()    

## Lastly, network clustering
`networkx` is a general purpose network analysis library. `python-louvain` is a network clustering algorithm

Read more on [Louvain and Leiden](https://www.nature.com/articles/s41598-019-41695-z) algorithms

In [ ]:
import networkx as nx
import community

## Create a network of correlation between patients
### Remove edges with relatively low correlation to simplify the data

In [ ]:
plt.hist(crc_corr_matrix.to_numpy().flatten(), bins = 20)
plt.xlabel('correlation'); plt.ylabel('frequency');

In [ ]:
crc_corr_matrix_trunc = crc_corr_matrix.to_numpy().copy()
crc_corr_matrix_trunc[crc_corr_matrix_trunc < 0.93] = 0
crc_corr_matrix_trunc -= np.eye(crc_corr_matrix_trunc.shape[0]) ## remove diagonal entries
    
crc_corr_network = nx.from_numpy_array(crc_corr_matrix_trunc)
crc_corr_network = nx.relabel_nodes(crc_corr_network, lambda x: crc_gene_data.index[x]) ## Add patient names to nodes
_ = crc_corr_network.edges(data = True)

### Visualize a random subnetwork
Don't worry about the codes

In [ ]:
np.random.seed(4649)
patients = np.random.choice(crc_gene_data.index, size = 200, replace = False)
patients = max(nx.connected_components(crc_corr_network.subgraph(patients)), key = len)
selected_graph = crc_corr_network.subgraph(patients)
_ = selected_graph.edges(data = True)

plt.figure(figsize = (8, 8))
edges, weights = zip(*nx.get_edge_attributes(selected_graph, 'weight').items())
nx.draw_spring(selected_graph, node_size = 40, edgelist = edges, edge_color = weights, edge_cmap = plt.cm.Greys, alpha = 0.8)
plt.show()

## Use Louvain algorithm to identify partitions that maximize modularity

In [ ]:
partition = community.best_partition(crc_corr_network, random_state = 4649)

In [ ]:
louvain_cluster = np.array([partition[x] for x in crc_gene_data.index])
unique, counts = np.unique(louvain_cluster, return_counts = True)

plt.figure(figsize = (5, 3))
plt.hist(counts, bins = 20)
plt.xlabel('Partition size'); plt.ylabel('Number of nodes')
plt.show()

### Visualize only the large clusters

In [ ]:
def view_large_clusters(clusters, size_cutoff, embeds = [crc_gene_final_embed, crc_gene_pcoa_embed]):
    outliers = []
    
    plt.figure(figsize = (10, 5))
    plt.subplot(1, 2, 1)

    for k in np.unique(clusters):
        filt = (clusters == k)
        
        if filt.sum() >= size_cutoff:
            plt.scatter(embeds[0][filt, 0], embeds[0][filt, 1], s = 20, label = 'Cluster ' + str(k))
        else:
            outliers.append(k)

    filt = [label for label in clusters if label in outliers]
    plt.scatter(embeds[0][filt, 0], embeds[0][filt, 1], s = 20, label = 'Outliers', c = 'tab:gray', alpha = 0.3)       
    plt.xlabel('UMAP1'); plt.ylabel('UMAP2'); plt.legend()

    plt.subplot(1, 2, 2)

    for k in np.unique(clusters):
        filt = (clusters == k)
        
        if filt.sum() >= size_cutoff:
            plt.scatter(embeds[1][filt, 0], embeds[1][filt, 1], s = 20, label = 'Cluster ' + str(k))

    filt = [label for label in clusters if label in outliers]
    plt.scatter(embeds[1][filt, 0], embeds[1][filt, 1], s = 20, label = 'Outliers', c = 'tab:gray', alpha = 0.3)
    plt.xlabel('PC1'); plt.ylabel('PC2'); plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
view_large_clusters(louvain_cluster, 20)

## Check the distribution of clinical data across clusters

In [ ]:
updated_louvain_cluster = np.empty(louvain_cluster.shape, dtype = 'object')
updated_louvain_cluster[:] = 'Outliers'
updated_louvain_cluster[louvain_cluster == 5] = 'Cluster 1'
updated_louvain_cluster[louvain_cluster == 8] = 'Cluster 2'
updated_louvain_cluster[louvain_cluster == 9] = 'Cluster 3'

In [ ]:
crc_clin_data.columns

In [ ]:
for feature in ['gender', 'stage', 'tStage', 'nStage', 'mStage', 'tumorLocation', 'microsatelite']:
    sns.heatmap(pd.crosstab(crc_data[feature], updated_louvain_cluster, normalize = 'columns'))
    plt.show()

## Extra technique - unsupervised feature selection
The majority of 20,293 genes measured here would NOT be related to colorectal cancer. So the similiarity calculated using all of the could be misleading

### Screen for highly variable genes

In [ ]:
plt.scatter(crc_gene_data.mean(), crc_gene_data.std())
plt.xlabel('average expression'); plt.ylabel('standard deviation')
plt.show()

### A lazy method is to sort genes by standard deviation (regardless of average expression)
`np.argsort()` returns the indices of genes ordered by standard deviation from low to high

In [ ]:
highly_variable_genes = crc_gene_data.columns[np.argsort(crc_gene_data.std())[-2000:]]

plt.scatter(crc_gene_data.mean(), crc_gene_data.std())
plt.scatter(crc_gene_data.mean()[highly_variable_genes], crc_gene_data.std()[highly_variable_genes])
plt.xlabel('average expression'); plt.ylabel('standard deviation')
plt.show()

### Impact of gene selection on patient-patient correlation

In [ ]:
crc_corr_matrix_hv = crc_gene_data[highly_variable_genes].T.corr(method = 'pearson')
display(crc_corr_matrix_hv.head(3))

In [ ]:
plt.hist(crc_corr_matrix_hv.to_numpy().flatten(), bins = 20)
plt.xlabel('correlation'); plt.ylabel('frequency');

## Re-run the dimensionality reduction techniques

In [ ]:
crc_gene_pca_embed_hv = PCA(random_state = 4649).fit_transform(crc_gene_data_std[highly_variable_genes])
crc_gene_pcoa_embed_hv = pcoa.fit_transform(1 - crc_corr_matrix_hv)
crc_gene_umap_embed_hv = umap.UMAP(n_neighbors = 50, min_dist = 0.25, random_state = 4649).fit_transform(crc_gene_data_std[highly_variable_genes])

In [ ]:
plt.figure(figsize = (14, 4))
plt.subplot(1, 3, 1); plt.scatter(crc_gene_pca_embed_hv[:, 0], crc_gene_pca_embed_hv[:, 1]); plt.title('PCA')
plt.subplot(1, 3, 2); plt.scatter(crc_gene_pcoa_embed_hv[:, 0], crc_gene_pcoa_embed_hv[:, 1]); plt.title('PCoA with correlation')
plt.subplot(1, 3, 3); plt.scatter(crc_gene_umap_embed_hv[:, 0], crc_gene_umap_embed_hv[:, 1]); plt.title('UMAP')
plt.tight_layout()
plt.show()

In [ ]:
view_embed_clinical_cat(crc_gene_pca_embed_hv, 'tumorLocation')
view_embed_clinical_cat(crc_gene_pca_embed_hv, 'microsatelite', ['MSS', 'MSI-L', 'MSI-H', 'Indeterminate'])

In [ ]:
view_embed_clinical_cat(crc_gene_pcoa_embed_hv, 'tumorLocation')
view_embed_clinical_cat(crc_gene_pcoa_embed_hv, 'microsatelite', ['MSS', 'MSI-L', 'MSI-H', 'Indeterminate'])

In [ ]:
view_embed_clinical_cat(crc_gene_umap_embed_hv, 'tumorLocation')
view_embed_clinical_cat(crc_gene_umap_embed_hv, 'microsatelite', ['MSS', 'MSI-L', 'MSI-H', 'Indeterminate'])

## Re-perform network clustering

In [ ]:
crc_corr_matrix_trunc_hv = crc_corr_matrix_hv.to_numpy().copy()
crc_corr_matrix_trunc_hv[crc_corr_matrix_trunc_hv < 0.65] = 0
crc_corr_matrix_trunc_hv -= np.eye(crc_corr_matrix_trunc_hv.shape[0]) ## remove diagonal entries
    
crc_corr_network_hv = nx.from_numpy_array(crc_corr_matrix_trunc_hv)
crc_corr_network_hv = nx.relabel_nodes(crc_corr_network_hv, lambda x: crc_gene_data.index[x]) ## Add patient names to nodes
_ = crc_corr_network_hv.edges(data = True)

In [ ]:
partition_hv = community.best_partition(crc_corr_network_hv, random_state = 4649)

In [ ]:
louvain_cluster_hv = np.array([partition_hv[x] for x in crc_gene_data.index])
unique_hv, counts_hv = np.unique(louvain_cluster_hv, return_counts = True)

plt.figure(figsize = (5, 3))
plt.hist(counts_hv, bins = 20)
plt.xlabel('Partition size'); plt.ylabel('Number of nodes')
plt.show()

### Visualize the new result

In [ ]:
view_large_clusters(louvain_cluster_hv, 20, embeds = [crc_gene_umap_embed_hv, crc_gene_pcoa_embed_hv])

### With clinical data

In [ ]:
updated_louvain_cluster_hv = np.empty(louvain_cluster_hv.shape, dtype = 'object')
updated_louvain_cluster_hv[:] = 'Outliers'
updated_louvain_cluster_hv[louvain_cluster_hv == 2] = 'Cluster 1'
updated_louvain_cluster_hv[louvain_cluster_hv == 5] = 'Cluster 2'
updated_louvain_cluster_hv[louvain_cluster_hv == 8] = 'Cluster 3'
updated_louvain_cluster_hv[louvain_cluster_hv == 9] = 'Cluster 4'

In [ ]:
for feature in ['gender', 'stage', 'tStage', 'nStage', 'mStage', 'tumorLocation', 'microsatelite']:
    sns.heatmap(pd.crosstab(crc_data[feature], updated_louvain_cluster_hv, normalize = 'columns'))
    plt.xlabel('')
    plt.show()